In [78]:
import time
import numpy as np
from math import *
import pandas as pd


In [79]:
# %%time
#---------------------------------导入文件-------------------------
data_cell = pd.read_csv(open('ee.csv'),iterator=True)
data_much = pd.read_csv(open('rr.csv'))
display(data_cell.get_chunk(1),data_much.iloc[0:1,])

,小区CGI(*),RRU经度,RRU纬度,方位角,enodeb
0,460-00-641901-2,113.74424,30.89762,120,641901


,cgi-2,RRU经度-2,RRU纬度-2,enodeb
0,460-00-585371-130,113.918335,30.96527,585371


In [80]:
%%time
#---------------------------------数据处理-------------------------
#----导入信息模版----('地市','表1经度','表1纬度','表1方位角','表2经度','表2纬度')
table_city, table_1_lon, table_1_lat, table_1_fw, table_2_lon, table_2_lat = ('enodeb','RRU经度','RRU纬度','方位角','RRU经度-2','RRU纬度-2')
jiao, juli_max, juli_min = (60 ,50,0)
#-----轮换计算条件-----
loop = True
chunkSize = 500
chunks = pd.DataFrame()
i = 0
# aa = 0
while loop:
    try:
        chunk = data_cell.get_chunk(chunkSize)
        ss = pd.merge(chunk ,data_much , how = 'inner',on=table_city)
        #-------判断是否在距离范围内
        lon1, lat1, lon2, lat2 = map(np.radians, [ss[table_1_lon], ss[table_1_lat], ss[table_2_lon], ss[table_2_lat]])
        a = np.sin((lat2 - lat1)/2)**2 + np.cos(lat1)*np.cos(lat2)*np.sin((lon2 - lon1)/2)**2
        ss['距离'] = 2*np.arcsin(np.sqrt(a))*6371*1000
        #--------判断是否在方位角范围内
        radLonA, radLatA, radLonB, radLatB = map(np.radians, [ss[table_1_lon], ss[table_1_lat], ss[table_2_lon], ss[table_2_lat]])
        x = np.cos(radLatA) * np.sin(radLatB) - np.sin(radLatA) * np.cos(radLatB) * np.cos(radLonB - radLonA)
        brng = np.degrees(np.arctan2((np.sin(radLonB - radLonA) * np.cos(radLatB)), x))
        ss['方位角差异'] =(brng + 360) % 360
#         #---------以下是判断条件
#         ss = ss.loc[((np.abs(ss[table_1_fw]-ss['方位角'])<jiao) | (360-np.abs(ss[table_1_fw]-ss['方位角'])<jiao)) \
#                     & ((ss['距离']>juli_min) & (ss['距离']<juli_max))]
        

        #--------导入到表中累计
        chunks = chunks.append(ss)
#         i+=1
#         if i == 1:
#             break
    except StopIteration:
        loop = False
        print("Iteration is stopped.")

Iteration is stopped.
Wall time: 12 ms


In [81]:
print(chunks.shape)
chunks.to_csv('ttts1.csv',encoding='gbk')
chunks.iloc[0:6,]

(239, 10)


,小区CGI(*),RRU经度,RRU纬度,方位角,enodeb,cgi-2,RRU经度-2,RRU纬度-2,距离,方位角差异
0,460-00-456728-6,113.923590,30.679220,300.0,456728,460-00-456728-7,113.923590,30.679220,0.0,0.0
1,460-00-456728-6,113.923590,30.679220,300.0,456728,460-00-456728-8,113.923590,30.679220,0.0,0.0
2,460-00-456728-6,113.923590,30.679220,300.0,456728,460-00-456728-4,113.923590,30.679220,0.0,0.0
3,460-00-456728-6,113.923590,30.679220,300.0,456728,460-00-456728-9,113.923590,30.679220,0.0,0.0
4,460-00-641935-6,114.112259,31.568995,240.0,641935,460-00-641935-8,114.112259,31.568995,0.0,0.0
5,460-00-641935-6,114.112259,31.568995,240.0,641935,460-00-641935-9,114.112259,31.568995,0.0,0.0
